# CSI 4142 - Introduction to Data Science
# Assignment 2: Data cleaning

Shacha Parker (300235525)\
Callum Frodsham and (300199446)\
Group 79

### Setup Instructions To Reproduce this Data Cleaning Notebook:
(Step 1 Optional)
1. Create a virtual python environment in the project directory (if you want) for all of the packages required:  
``` 
python -m venv .venv
```
To enter the virutal environment: 
```
.venv/Scripts/activate.ps1 # on windows
source .venv/bin/activate # on mac/linux
```
2. Download all of the required packages (run in cmd/shell of choice):
```
pip install jupyter
pip install ipykernel
pip install pandas
pip install numpy
```
3. VSCode: Ensure you have the correct python kernel selected!
<br> 
If you are using a virtual environment, make sure to select the python interpreter for that virtual environment otherwise this will not work! If you have everything done globally, then just make sure the correct python kernel you are using is selected.

In [ ]:
# Initial imports
import numpy as np
import pandas as pd

<h1>Introduction: </h1>
<p>This data error checking notebook is about exploring a dataset, specifically Shivam Bansal's Netflix dataset, and evaluating all of its features, and checking for different types of dataset validity errors. These errors include, look-up errors, range errors, duplicate errors, uniqueness errors, etc.</p>

<h1>Dataset 1: Netflix Movies and TV Shows</h1>
<h3>Clean Data Checking</h3>

Author: Shivam Bansal
<br>
Purpose: This dataset was made to provide insights on the shows and movies that Netflix is hosting on their platform. For example, these insights could be used to see what type of content the platform is missing, or what type of content they have too much of.
<br>
Shape: This dataset is composed of 12 columns, and 8810 rows.
<br><br>
Link: <a href="https://www.kaggle.com/datasets/shivamb/netflix-shows"> Mobile Device Usage and User Behavior</a>
<br>
<h3>Dataset Feature List: </h3>
<ol>
    <li>Show Id:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The show_id is a unique ID that is assigned to each show/movie. There are 8807 entries ranging from s1 to s8807.
    </li>
    <br>
    <li>Type
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description Informs the viewer whether the content is a tv show or a movie.
        </li>
    <br>
    <li>Title:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The title name of the tv show or movie.
        </li>
    <br>
    <li>Director:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The name of the person(s) who directed the tv show or movie. 
        </li>
    <br>
    <li>Cast:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The name(s) of the notable actor(s) who acted in the tv show or movie.
        </li>
    <br>
    <li>Country:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The countries/country where the tv show or movie was produced.
        </li>
    <br>
    <li>Date Added:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The date the show or movie was added to Netflix.
        </li>
    <br>
    <li>Release Year:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The year in which the tv show or movie was originally released.
        </li>
    <br>
    <li>Rating:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: This rating indicates the acceptable age of viewing for the tv show or movie.
        </li>
    <br>
    <li>Duration:
    <br>
    Feature Type: Mixed Type - Numerical Continuous - Categorical Ordinal 
    <br>
    Description: The duration of the movie in minutes, or if it is a tv show, in seasons.
        </li>
    <br>
    <li>Genre/Listed In:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The genres/subgenres the tv show or movie falls in.
        </li>
    <br>
    <li>Description:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The description of the tv show or movie.
        </li>
</ol>

In [ ]:
# load the dataset: 
dataset = pd.read_csv("https://raw.githubusercontent.com/CLFrod/Assignment2CSI4142/refs/heads/master/netflix_titles.csv")

<h4>1. Range Check:</h4>
<p>
In this test, we will verify the range of a numerical value. The range is the minimum and maximum values that an attribute can have.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
range_attributes = ['release_year', 'date_added']
# attribute selection:
test_range_attribute = 'release_year'
# Minimum:
range_minimum = 1888
# Maximum:
range_maximum = 2025

In [ ]:
# Checker Code
range_series = dataset[test_range_attribute].between(range_minimum, range_maximum)

# changing the booleans in range_series from True to False and vice versa
# values that were previously not in range (False) are true!
not_range_series = ~range_series

# check if the range series is empty
empty_range = dataset[not_range_series].empty
# if the series is not empty, 
if not empty_range:
    out_of_range_count = 0
    for val in not_range_series:
        out_of_range_count += 1
    print(out_of_range_count)
# if the series is empty
else:
    print("All values fall within specified paramters!")

<p style ="font-size:20px">Range check Findings: </p>
No range errors were detected based on the provided parameters.
The dataset's 'release_year' feature only has years that fit within logical parameters.

<h4>2. Format Check:</h4>
<p>
In this test, we will verify whether or not specifc values adhere to a specifc format. 
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
format_attributes = ['date_added', 'release_year']

test_format_attribute = format_attributes[0]


In [ ]:
# Checker Code
# get the df of dates added
date_added_df = dataset['date_added']

#convert to datetime
converted_dates = pd.to_datetime(date_added_df, format="%B %d, %Y", errors="coerce")

# get the incorrectly formatted dates
incorrectly_formatted_dates = converted_dates.isna()
print(dataset[incorrectly_formatted_dates][test_format_attribute])


<p style ="font-size:20px">Format check Findings: </p>
There are 98 data points that don't follow the correct format of the dates,
For example,
Row 6068 is missing a date, and thus is technically not following the correct format.<br>
<br>
Row 8759 has " November 1, 2016". Although technically following correct formatting, it has a leading whitespace which should not be there.

<h4>3. Data Type Check:</h4>
<p>
In this test, we will verify whether or not an attribute's values conform to the specific data type.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
attributes = ['release_year']
data_type_test_attribute = attributes[0]

# small function to check if val is int
def is_integer(val):
    return isinstance(val, int)

In [ ]:
# Checker Code


data_type_check = dataset[data_type_test_attribute].apply(is_integer).all()

if data_type_check:
    print("All values are integers.")
else:
    print("Not all values are integers.")


<p style ="font-size:20px">Data Type check Findings: </p>
No data type errors were detected based on the provided parameters.
The dataset's 'release_year' feature only contains integer values.

<h4>4. Consistency Check</h4>
<p>
In this test, we will verify if two values are consistent with one another. For example, the release_year coming before the date added to a platform.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
consistency_attributes = ['release_year', "date_added"]


In [ ]:
# Checker Code
# deep copy the dataset
temp_df = dataset.copy(True)
# convert date added to python datetime
temp_df[consistency_attributes[1]] = pd.to_datetime(temp_df[consistency_attributes[1]], errors='coerce')
# get the year only
temp_df['year_added'] = temp_df[consistency_attributes[1]].dt.year

# get the dates that aren't consistent
inconsistent_dates = temp_df[temp_df['year_added'] < dataset[consistency_attributes[0]]]

print(inconsistent_dates[consistency_attributes[0]])
print(inconsistent_dates.size)

<p style ="font-size:20px">Consistency check Findings: </p>
There are 182 data points that are not logically consistent with the date added, and the release year.

For example,

The Film "Hilda" on row 1553 was added in 2020, but the release year says 2021. That should not be possible, unless the series was added to Netflix as a test beforehand or for some other reason.


<h4>5. Uniqueness Check:</h4>
<p>
In this test, we will be performing a check to see if all values in a column are unique.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
uniqueness_attributes = ['show_id','title']

In [ ]:
# Checker Code

if dataset[uniqueness_attributes[0]].is_unique:
    print("All " + uniqueness_attributes[0] + "s are unique values.")
else:
    print("All " + uniqueness_attributes[0] + "s are not unique values.")



<p style ="font-size:20px">Uniqueness check Findings: </p>
No uniqueness errors were detected based on the provided parameters!

<h4>6. Presence Check:</h4>
<p>
In this test, we will be checking if an attribute has any missing, or null values present in the dataset.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
presence_attributes = ['title', 'date_added', 'director']

In [ ]:
# Checker Code
# get 
presence_series = dataset[presence_attributes[1]].isnull()

print("Rows with missing "+ presence_attributes[1] + ":")
print(dataset[presence_series]['title'])

<p style ="font-size:20px">Presence check Findings: </p>
There are 10 data points in the dataset where the date_added, is empty or null. See for example the following rows:

show_id / title / date_added / release_year
s6067 / A Young Doctor's Notebook and Other Stories / / 2013


<h4>7. Length Check:</h4>
<p>
In this test, we will be verifying if the length of a value conforms to a possible set max length.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
length_attributes = ['release_year', 'rating']
test_length_attribute = length_attributes[1]
# max length of tv rating
max_length = 8

In [ ]:
# Checker Code
# check if length falls within bounds
length_check_df = dataset[dataset[test_length_attribute].str.len() > max_length]

print(length_check_df['rating'])

<p style ="font-size:20px">Length check Findings: </p>
No length errors were detected based on the provided parameters!

<h4>8. Look-up Check:</h4>
<p>
In this test we are checking if a value conforms to the set values in a look-up table. The look-up table contains all of the values an attribute should be. 
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
look_up_attributes = ['rating', 'type']

test_look_up_attribute = look_up_attributes[1]

In [ ]:
# Checker Code

look_up_table = ['TV Show', 'Movie']

# Check which values are valid
valid_content_types = dataset['type'].isin(look_up_table)

print(dataset[~valid_content_types])


<p style ="font-size:20px">Look-up check Findings: </p>
No look-up errors were found for the 'type' feature. Thus, all values in the column 'type' conform to the look-up table.

<h4>9. Exact Duplicate Check</h4>
<p>
In this test we are checking for exact duplicate values.
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
duplicate_attributes = ['title']


In [ ]:
# Checker Code
# check if any values are exact duplicates in the title
duplicate_series = dataset[duplicate_attributes[0]].duplicated()

print(dataset[duplicate_series])

<p style ="font-size:20px">Exact Duplicate check Findings: </p>
No exact duplicate errors were found for the 'title' feature. Thus, all values in the column 'title' are unique.

<h4>10. Near Duplicate Check:</h4>
<p>
In this test, we will be checking to see if there are any near-duplicates in a column. This test has been modified due to the fact that there are no duplicate rows (or near duplicate rows), as well as there are no significant duplicate values in this dataset. 
</p>

In [ ]:
# Please enter the various attributes below to perform the tests:
n_duplicate_attributes = ['date_added']


In [ ]:
# Checker Code
# get all date_added values with a leading whitespace
no_nan_date_added = dataset[dataset['date_added'].notna()]

# Get all dates with a leading whitespace! :)
all_dates_leading_whitespace = no_nan_date_added[no_nan_date_added['date_added'].str.startswith(' ')].copy(deep=True)

# Get all dates without a leading whitespace!
all_dates_without_leading_whitespace = no_nan_date_added[~no_nan_date_added['date_added'].str.startswith(' ')].copy(deep=True)

# strip all whitespaces from leading whitespace group to make it easier to compare:
all_dates_leading_whitespace_fixed = all_dates_leading_whitespace['date_added'].str.lstrip()

# now check for duplicates now that we have both groups:
duplicates_across_groups = all_dates_leading_whitespace[
    all_dates_leading_whitespace_fixed.isin(all_dates_without_leading_whitespace['date_added'])
]
print(duplicates_across_groups['date_added'])


<p style ="font-size:20px">Near-Duplicate check Findings: </p>
There are 73 data points/examples of values within the date_added columnn where the only difference between the two values are a leading whitespace, making them near-duplicates of eachother. (This was chosen since there are no significant near-duplicates)

An example two rows of near-duplicate values:
Row 6081, and row 6615:
Row 6081 has the date_added = " August 4, 2017", while row 6615 has the date_added = "August 4, 2017". These valuse are nearly duplicate entries, the only difference being the leading whitespace. 

<h2>Conclusion:</h2>
Overall this dataset had many errors, such as date format errors where the dates were input with leading whitespaces. There were consistency errors where tv content was added to the platform before the content had even released. There were presence errors found, where rows are missing the dates they were added to the platform. There were many errors found within this dataset, but there were also many sections without proper errors, such as the near-duplicate error category where the goal of finding a near-duplicate error had to be interpreted differently to find somewhat meaningful values. 

<h3>References:</h3>
<ul>
<li>
<a href="https://www.w3schools.com/python/python_datetime.asp"> Python Date time formatting</a>
</li>
<li>
<a href="https://stackoverflow.com/questions/402504/how-to-determine-a-python-variables-type"> Check Variable Type</a>
</li>
<li>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html"> "isin()" Pandas Documentation</a>
</li>
<li>
<a href="https://saturncloud.io/blog/how-to-select-data-from-a-pandas-dataframe-using-startswith/#:~:text=startswith()%20method%20provided%20by,the%20rows%20of%20the%20Dataframe."> Check if row value starts with a specified value.</a>
</li>


</ul>